1. [Introduction:](#1)
    1. [대회 개요 및 미국 부동산 특징](#2)
    1. [사용 Library](#3)
1. [데이터 확인](#4)
    1. [데이터 셋 확인](#5)
    1. [데이터 변수 정의](#6)
    1. [데이터 변수명 변경](#7)
1. [EDA](#8)
    1. [전체 데이터 개요](#9)
    1. [House 데이터](#10)
    1. [Location 데이터](#11)
    1. [Finance 데이터](#12)
1. [데이터 변환](#13)
    1. [Null Data 처리](#14)
    1. [파생변수 생성](#15)
    1. [One-hot Encoding](#16)
1. [모델링 및 평가](#35)
    1. [GLM](#36)
    1. [Light Gradient Boosting](#37)
1. [결론](#38)
    1. [모델 선정](#39)
    1. [추가적인 접근법](#40)

<a id="1"></a> <br>
# 1. INTRODUCTION

<a id="2"></a> <br>
# 1.1 대회 개요


**Zestimate**는 각 재산에 대한 수백 개의 데이터 포인트를 분석하는 750만 개의 통계 및 기계 학습 모델을 기반으로 가정 값을 추정한다. 그리고, 오류의 중간 마진을 지속적으로 개선함으로써(초기의 14%에서 오늘 날의 5%로), zillow는 그 후 미국에서 가장 크고 가장 신뢰할 수 있는 부동산 정보 시장 중 하나이자 영향력 있는 기계 학습의 선도적인 예로 자리매김하게 되었다. 예선에서는 잔차 오류를 개선하기 위한 모델을 작성할 것이다. 최종 라운드에서는 외부 데이터 소스를 사용하여 모델에 경쟁 우위를 부여하는 새로운 기능을 설계하는 데 도움이 되는 홈 평가 알고리즘을 처음부터 구축할 것이다.

**Objective:**
잔차 추정 오류를 개선하기 위한 모델 작성.

**Zillow:**
Zillow 는 2006년에 설립된 온라인 부동산 데이터베이스 회사 - 위키백과

**Zestimate:**
"Zestimate"는 각 재산에 대한 수백 개의 데이터 포인트를 분석하는 750만 개의 통계 및 기계 학습 모델을 기반으로 가정 값을 추정한다. Zestimate는 접근할 수 있는 주택에 대한 모든 정보를 종합하여 데이터를 획득한다. 다양한 소스의 데이터를 하나로 혼합하고 병합한다. 온마켓 데이터 - 사각형 화면, 위치 또는 욕실 수를 포함한 주택 특성, 단단한 목재 바닥, 화강암 카운터톱 또는 조경된 뒷마당 같은 독특한 특징, 해당 지역의 가격, 설명, 비교 가능한 주택 및 시판 일수, 오프마켓 데이터 - 세금 평가, 이전 판매 및 기타 공개적으로 사용 가능한 기록

미국 부동산 특징


미국 부동산의 가격을 결정하는 요인으로 6가지를 대표적으로 꼽을 수 있다.
- 과거 판매 가격
- 이웃
- 상점
- 크기 및 외형
- 연식 및 상태
- 주변 특징

<a id="3"></a> <br>
# 1.2 사용 Library 

In [ ]:
# For DataFrame
import numpy as np # for linear algebra
import pandas as pd # for data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

# For Analysis
from sklearn import neighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


# For Visualization
import plotly # visualization
from plotly.graph_objs import Scatter, Figure, Layout # visualization
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot # visualization
import plotly.figure_factory as ff # visualization
import plotly.graph_objs as go # visualization
init_notebook_mode(connected=True) # visualization
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for data visualization
color = sns.color_palette()
%matplotlib inline


<a id="4"></a> <br>
# 2. 데이터 확인

<a id="5"></a> <br>
# 2.1 데이터 셋 확인

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

Train Data Set

In [ ]:
train_2016_df = pd.read_csv("../input/train_2016_v2.csv", parse_dates=["transactiondate"])
train_2016_df.shape

In [ ]:
train_2016_df.head()

In [ ]:
train_2017_df = pd.read_csv("../input/train_2017.csv", parse_dates=["transactiondate"])
train_2017_df.shape

In [ ]:
train_2017_df.head()

Properties Data Set

In [ ]:
prop_2016_df = pd.read_csv("../input/properties_2016.csv")
prop_2016_df.shape

In [ ]:
prop_2016_df.head()

In [ ]:
prop_2017_df = pd.read_csv("../input/properties_2017.csv")
prop_2017_df.shape

In [ ]:
prop_2017_df.head()

Merge

In [ ]:
merge_2016_df = pd.merge(train_2016_df,prop_2016_df,on="parcelid",how="left")
merge_2016_df.shape

In [ ]:
merge_2016_df.head()

In [ ]:
merge_2017_df = pd.merge(train_2017_df,prop_2017_df,on="parcelid",how="left")
merge_2017_df.shape

In [ ]:
missingvalues_prop = (merge_2017_df.isnull().sum()/len(merge_2017_df)).reset_index()
missingvalues_prop.columns = ['field','proportion']
missingvalues_prop = missingvalues_prop.sort_values(by = 'proportion', ascending = False)
print(missingvalues_prop)

In [ ]:
merge_2016_df.head()

In [ ]:
merge_2016_df.dtypes

<a id="6"></a> <br>
# 2.2 데이터 변수 정의

* 0 parcelid
* logerror
* transactiondate : 거래 일 
* transaction_month : 거래 월 
* airconditioningtypeid : 냉방 시스템 종류
    
    [Category] 
    1 Centra 
    2 Chilled Water 
    3 Evaporative Cooler 
    4 Geo Thermal 
    5 None 
    6 Other 
    7 Packaged AC Unit 
    8 Partial 
    9 Refrigeration 
    10 Ventilation 
    11 Wall Unit 
    12 Window Unit 
    13 Yes 
* architecturalstyletypeid : 건물 건축 양식 

    [Category] 
    1 A-Frame 
    2 Bungalow 
    3 Cape Cod 
    4 Cottage 
    5 Colonial 
    6 Custom 
    7 Contemporary 
    8 Conventional 
    9 Dome 
    10 French Provincial 
    11 Georgian 
    12 High Rise 
    13 Historical 
    14 Log Cabin/Rustic 
    15 Mediterranean 
    16 Modern 
    17 Mansion 
    18 English 
    19 Other 
    20 Prefab 
    21 Ranch/Rambler 
    22 Raised Ranch 
    23 Spanish 
    24 Traditional 
    25 Tudor 
    26 Unfinished/Under Construction 
    27 Victorian 
* basementsqft : 지하실 
* bathroomcnt : 화장실 갯수 
* bedroomcnt : 침실 갯수
* buildingclasstypeid : 건물의 뼈대 구조 

    [Category] 
    1 Buildings having fireproofed structural steel frames carrying all wall, floor and roof loads. Wall, floor and roof structures are built of non-combustible materials. 
    2 Buildings having fireproofed reinforced concrete frames carrying all wall floor and roof loads which are all non-combustible. 
    3 Buildings having exterior walls built of a non-combustible material such as brick, concrete, block or poured concrete. Interior partitions and roof structures are built of combustible materials. Floor may be concrete or wood frame. 
    4 Buildings having wood or wood and steel frames 
    5 Specialized buildings that do not fit in any of the above categories 
* buildingqualitytypeid : 건물의 전반적인 상태 평가 점수
* calculatedbathnbr : 화장실 갯수 
* decktypeid : 데크 타입 
* finishedfloor1squarefeet : 1층 전용면적 
* calculatedfinishedsquarefeet : 전체 전용면적 
* finishedsquarefeet12 : Finished living area
* finishedsquarefeet13 : Perimeter living area
* finishedsquarefeet15 : Total area
* finishedsquarefeet50 :  Size of the finished living area on the first (entry) floor of the home
* finishedsquarefeet6 : Base unfinished and finished area
* fips : 연방정보처리표준코드
* fireplacecnt : 가정 내 벽난로 수(있는 경우)
* fullbathcnt : 가정 내 full 욕실 수(싱크, 샤워기 + 욕조, 화장실)
* garagecarcnt : 부착된 차고지를 포함한 총 차고 수
* garagetotalsqft : 부착된 차고지를 포함하여 모든 차고 총 제곱 피트 수
* hashottuborspa : 집에 뜨거운 욕조나 온천이 유무
* heatingorsystemtypeid : 가정용 난방 시스템 유형
* latitude
* longitude
* lotsizesquarefeet : 대지의 면적
* poolcnt : lot(대지) 내의 수영장 수
* poolsizesum : 부동산에 있는 모든 수영장의 총 제곱 크기.
* pooltypeid10 :  Spa or Hot Tub
* pooltypeid2 :  Pool with Spa/Hot Tub
* pooltypeid7 :  Pool without hot tub
* propertycountylandusecode : 카운티 토지 이용 코드, 즉 카운티 수준에서 구역 지정이다.
* propertylandusetypeid : 토지 유형
* propertyzoningdesc : 해당 재산에 대해 허용된 토지 이용(구역 설정)에 대한 설명.
* rawcensustractandblock : 센서스와 블록 ID 결합 - 확장별 블록 그룹 할당도 포함
* regionidcity : 부동산이 있는 도시
* regionidcounty : 부동산이 위치한 카운티
* regionidneighborhood : 부동산이 있는 이웃
* regionidzip : 부동산이 있는 우편번호
* roomcnt : 부동산에 있는 총 방 수
* storytypeid : 다층 주택의 층 유형(즉, 지하 및 주층, 분할층, 다락원 등)
* threequarterbathnbr : 3/4 화장실(샤워기 + 세면대 + 변기) 갯수
* typeconstructiontypeid : 어떤 종류의 건축자재가 그 집을 짓는데 사용되었는가0
* unitcnt : 구조물이 건축 된 단위 수(즉, 2 = 이중, 3 = 삼중 등)
* yardbuildingsqft17 : 마당의 파티오 면적
* yardbuildingsqft26 : 마당에 보관 창고/건물 면적
* yearbuilt : 주요 거주지가 건설된 해
* numberofstories : 집에 있는 story나 층수입니다.
* fireplaceflag : 이 집에 벽난로 유무
* structuretaxvaluedollarcnt : 소포 위에 세워진 구조물의 평가 값
* taxvaluedollarcnt : 구획의 총 세액 평가액
* assessmentyear : 재산세 과세 연도
* landtaxvaluedollarcnt : 구획의 토지 면적 평가 값
* taxamount : 그 평가 년도에 대해 평가된 총 재산세
* taxdelinquencyflag : 이 소포의 재산세는 2015 년까지 만기 인가?.
* taxdelinquencyyear : 미납 재산세가 납부되어야 할 해 
* censustractandblock : 센서스와 블록 ID 결합 - 확장별 블록 그룹 할당도 포함


<a id="7"></a> <br>
# 2.3 데이터 변수명 및 type 변경

In [ ]:
# Rename columns
merge_2016_df = merge_2016_df.rename(columns = {
  "parcelid" : "id_parcel",
  "yearbuilt":"build_year",
  "basementsqft":"area_basement",
  "yardbuildingsqft17":"area_patio",
  "yardbuildingsqft26":"area_shed",
  "poolsizesum":"area_pool",
  "lotsizesquarefeet":"area_lot",
  "garagetotalsqft":"area_garage",
  "finishedfloor1squarefeet":"area_firstfloor_finished",
  "calculatedfinishedsquarefeet":"area_total_calc",
  "finishedsquarefeet6":"area_base",
  "finishedsquarefeet12":"area_live_finished",
  "finishedsquarefeet13":"area_liveperi_finished",
  "finishedsquarefeet15":"area_total_finished",
  "finishedsquarefeet50":"area_unknown",
  "unitcnt":"num_unit",
  "numberofstories":"num_story",
  "roomcnt":"num_room",
  "bathroomcnt":"num_bathroom",
  "bedroomcnt":"num_bedroom",
  "calculatedbathnbr":"num_bathroom_calc",
  "fullbathcnt":"num_bath",
  "threequarterbathnbr":"num_75_bath",
  "fireplacecnt":"num_fireplace",
  "poolcnt":"num_pool",
  "garagecarcnt":"num_garage",
  "regionidcounty":"region_county",
  "regionidcity":"region_city",
  "regionidzip":"region_zip",
  "regionidneighborhood":"region_neighbor",
  "taxvaluedollarcnt":"tax_total",
  "structuretaxvaluedollarcnt":"tax_building",
  "landtaxvaluedollarcnt":"tax_land",
  "taxamount":"tax_property",
  "assessmentyear":"tax_year",
  "taxdelinquencyflag":"tax_delinquency",
  "taxdelinquencyyear":"tax_delinquency_year",
  "propertyzoningdesc":"zoning_property",
  "propertylandusetypeid":"zoning_landuse",
  "propertycountylandusecode":"zoning_landuse_county",
  "fireplaceflag":"flag_fireplace",
  "hashottuborspa":"flag_tub",
  "buildingqualitytypeid":"quality",
  "buildingclasstypeid":"framing",
  "typeconstructiontypeid":"material",
  "decktypeid":"deck",
  "storytypeid":"story",
  "heatingorsystemtypeid":"heating",
  "airconditioningtypeid":"aircon",
  "architecturalstyletypeid":"architectural_style"
})

In [ ]:
merge_2016_df.head()

In [ ]:
# Rename columns
merge_2017_df = merge_2017_df.rename(columns = {
  "parcelid" : "id_parcel",
  "yearbuilt":"build_year",
  "basementsqft":"area_basement",
  "yardbuildingsqft17":"area_patio",
  "yardbuildingsqft26":"area_shed",
  "poolsizesum":"area_pool",
  "lotsizesquarefeet":"area_lot",
  "garagetotalsqft":"area_garage",
  "finishedfloor1squarefeet":"area_firstfloor_finished",
  "calculatedfinishedsquarefeet":"area_total_calc",
  "finishedsquarefeet6":"area_base",
  "finishedsquarefeet12":"area_live_finished",
  "finishedsquarefeet13":"area_liveperi_finished",
  "finishedsquarefeet15":"area_total_finished",
  "finishedsquarefeet50":"area_unknown",
  "unitcnt":"num_unit",
  "numberofstories":"num_story",
  "roomcnt":"num_room",
  "bathroomcnt":"num_bathroom",
  "bedroomcnt":"num_bedroom",
  "calculatedbathnbr":"num_bathroom_calc",
  "fullbathcnt":"num_bath",
  "threequarterbathnbr":"num_75_bath",
  "fireplacecnt":"num_fireplace",
  "poolcnt":"num_pool",
  "garagecarcnt":"num_garage",
  "regionidcounty":"region_county",
  "regionidcity":"region_city",
  "regionidzip":"region_zip",
  "regionidneighborhood":"region_neighbor",
  "taxvaluedollarcnt":"tax_total",
  "structuretaxvaluedollarcnt":"tax_building",
  "landtaxvaluedollarcnt":"tax_land",
  "taxamount":"tax_property",
  "assessmentyear":"tax_year",
  "taxdelinquencyflag":"tax_delinquency",
  "taxdelinquencyyear":"tax_delinquency_year",
  "propertyzoningdesc":"zoning_property",
  "propertylandusetypeid":"zoning_landuse",
  "propertycountylandusecode":"zoning_landuse_county",
  "fireplaceflag":"flag_fireplace",
  "hashottuborspa":"flag_tub",
  "buildingqualitytypeid":"quality",
  "buildingclasstypeid":"framing",
  "typeconstructiontypeid":"material",
  "decktypeid":"deck",
  "storytypeid":"story",
  "heatingorsystemtypeid":"heating",
  "airconditioningtypeid":"aircon",
  "architecturalstyletypeid":"architectural_style"
})

In [ ]:
merge_2017_df.head()

In [ ]:
merge_df_raw = merge_2016_df.append(merge_2017_df, ignore_index=True)
merge_df_raw.shape

In [ ]:
merge_df_raw.head()

In [ ]:
merge_df_raw[['aircon', 'architectural_style', 'num_bathroom', 'num_bedroom', 'framing', 'quality', 'num_bathroom_calc', 'deck', 'num_fireplace', 'num_bath','num_garage','flag_tub','heating','num_pool','pooltypeid10','pooltypeid2','pooltypeid7','zoning_landuse_county','zoning_landuse','zoning_property','region_city','region_county','region_neighbor','region_zip','num_room','story','num_75_bath','material','num_unit','build_year','num_story','flag_fireplace','tax_year','tax_delinquency_year']] = merge_df_raw[['aircon', 'architectural_style', 'num_bathroom', 'num_bedroom', 'framing', 'quality', 'num_bathroom_calc', 'deck', 'num_fireplace', 'num_bath','num_garage','flag_tub','heating','num_pool','pooltypeid10','pooltypeid2','pooltypeid7','zoning_landuse_county','zoning_landuse','zoning_property','region_city','region_county','region_neighbor','region_zip','num_room','story','num_75_bath','material','num_unit','build_year','num_story','flag_fireplace','tax_year','tax_delinquency_year']].astype(str)

<a id="8"></a> <br>
# 3. EDA

<a id="9"></a> <br>
# 3.1 전체 데이터 개요

In [ ]:
merge_df_raw.dtypes

In [ ]:
corrMatt = merge_df_raw.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True)

<a id="10"></a> <br>
# 3.2 House 데이터

건축년도별 LogError

In [ ]:
# LogErorr
df_train = merge_df_raw.copy()
df_train.loc[:,'abs_logerror'] = df_train['logerror'].abs()
worst_prediction = df_train['abs_logerror'].quantile(q=.95)


trace0 = go.Scatter(
    y = df_train[(df_train['fips']==6037)&(df_train['abs_logerror']>worst_prediction)].\
                groupby('build_year')['abs_logerror'].mean(),
    x = df_train[(df_train['fips']==6037)&(df_train['abs_logerror']>worst_prediction)].\
                groupby('build_year')['abs_logerror'].mean().index,
    mode = 'lines+markers',
    name = "Los Angeles", 
)
trace1 = go.Scatter(
    y = df_train[(df_train['fips']==6059)&(df_train['abs_logerror']>worst_prediction)].\
                groupby('build_year')['abs_logerror'].mean(),
    x = df_train[(df_train['fips']==6059)&(df_train['abs_logerror']>worst_prediction)].\
                groupby('build_year')['abs_logerror'].mean().index,
    mode = 'lines+markers',
    name = "Orange County"
)
trace2 = go.Scatter(
    y = df_train[(df_train['fips']==6111)&(df_train['abs_logerror']>worst_prediction)].\
                groupby('build_year')['abs_logerror'].mean(),
    x = df_train[(df_train['fips']==6111)&(df_train['abs_logerror']>worst_prediction)].\
                groupby('build_year')['abs_logerror'].mean().index,
    mode = 'lines+markers',
    name = "Ventura County"
)
data = [trace0, trace1, trace2]

plotly.offline.iplot(data, filename='line-mode')

### 부동산의 토지 사용 유형

In [ ]:
merge_df_raw['zoning_landuse'].unique()

31 Commercial/Office/Residential Mixed Used : 상업용/사무실/주거 복합용도개발 

47 Store/Office (Mixed Use) : 상점/사무실 복합용도개발

246 Duplex (2 Units, Any Combination): 듀플렉스 하우스(duplex house)는 한 개의 필지에 두 가구가 나란히 지어진 형태의 집을 말한다.

247 Triplex (3 Units, Any Combination) : 한 개의 필지에 세 가구가 나란히 지어진 형태의 집을 말한다.

248 Quadruplex (4 Units, Any Combination) : 한 개의 필지에 네 가구가 나란히 지어진 형태의 집을 말한다.

260 Residential General : 일반주거지역은 성장과 주택의 다양성이 예상되는 지역의 토지에 적용된다. 여기에는 단독주택, 다세대주택, 이동주택 등이 포함된다.

261 Single Family Residential : 단독주택

263 Mobile Home : 이동식 주택 (대출X)

264 Town House : 타운홈은 보통 2층이나 3층으로 되어 있어 한가족이 전층을 다 사용하지만 양 옆으로는 옆집과 붙어있다. 보통 타운홈은 게이트가 있는 단지내에 있어 PUD와 같이 여러 공용시설도 즐길 수 있고 집앞에는 잔디밭이 있고 바비큐를 할 수 있는 작은 마당이 있으나 자기 소유의 대지가 따로 있는 것은 아니다. 단지 건물에 대한 소유권이 있는 것이 PUD와 다른 점이다.

265 Cluster Home : 주거지들이 레크리에이션용으로 남겨진 공동 구역과 밀접하게 그룹화. 사실상 주민들은 매우 작은 야드를 가지고 있지만 넓은 공통의 공간을 즐길 수 있다.

266 Condominium : 콘도미니엄(Condominium)은 대형 부동산 단지를 개별 단위로 나누어 매각하는 것을 말한다. 소유권은 보통 콘도 관리에 의해 통제되는 특정 "공통 속성"에 대한 독점적 지분을 포함한다. 콘도미니엄 관리는 보통 잔디 정비나 제설 등 단지의 일상적인 운영을 보는 단위주들로 구성된다.

267 Cooperative : 협동주택, 공유집합 주택 등으로도 불린다. 코하우징은 프라이버시와 자신의 욕구를 충족시키면서 이웃과 협동생활을 하는 공동체 주거단지이다. 공동으로 이용하는 공동생활시설, 옥외공간 등의 공유시설이 갖추어져 있어 식사준비, 가정관리, 육아 등과 같은 일상적인 활동을 공동으로 한다.

269 Planned Unit Development :  게이트가 있는 단지를 조성하고 단지 내에 수영장이나 테니스코트, 또는 바베큐를 할 수 있는 넓직한 파티장소 같은 공용시설을 만들어 함께 사용하고 관리비를 모아 공동으로 관리하는 PUD(planned unit development)가 점점 늘어나는 추세이다. 단지에 게이트가 있어 방범이 특히 잘되어 있는 곳도 있고 게이트가 없는 경우도 있는데, PUD는 각 유닛마다 자기의 대지를 소유하고 있는 것이 특징이고 땅의 크기는 일반 단독주택보다는 작다.

275 Manufactured, Modular, Prefabricated Homes : 조립식 주택(대출 O)

In [ ]:
cnt_srs = merge_df_raw['zoning_landuse'].value_counts()
plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[3])
plt.xticks(rotation='vertical')
plt.xlabel('zoning_landuse', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.show()

아파트 =  266

비 아파트 = 246, 247, 248, 261, 264, 265, 267, 269, 31, 47, 260, 263, 275

In [ ]:
# 아파트 corr
df = merge_df_raw.loc[merge_df_raw['zoning_landuse']=='266.0']
corrMatt = df.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True)

In [ ]:
# 비 아파트 corr
df = merge_df_raw.loc[merge_df_raw['zoning_landuse']!='266.0']
corrMatt = df.corr()
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True)

연속형 변수

In [ ]:
continuous = ['area_basement','area_firstfloor_finished','area_total_calc','area_live_finished','area_liveperi_finished','area_total_finished','area_unknown','area_base','area_garage','area_lot','area_pool','area_patio','area_shed']
discrete = ['num_bathroom','num_bedroom','num_bathroom_calc','num_bath','num_garage','num_pool','num_room','num_75_bath','num_unit','num_story']
categories = ['aircon','architectural_style','framing','quality','deck','heating','pooltypeid2','pooltypeid7','story','material','build_year']


In [ ]:
### Continuous variable plots
for col in continuous:
    values = merge_df_raw[col].dropna()
    lower = np.percentile(values, 1)
    upper = np.percentile(values, 99)
    fig = plt.figure(figsize=(18,9));
    sns.distplot(values[(values>lower) & (values<upper)], color='Sienna', ax = plt.subplot(121));
    sns.boxplot(y=values, color='Sienna', ax = plt.subplot(122));
    plt.suptitle(col, fontsize=16)

이산형 변수

In [ ]:
merge_df_raw['aircon'].dropna().head(10)

In [ ]:
merge_df_raw[discrete] = merge_df_raw[discrete].astype(float)

In [ ]:
### Discrete variable plots
NanAsZero = ['num_pool', 'num_75_bath']
for col in discrete:
    if col in NanAsZero:
        merge_df_raw[col].fillna(0, inplace=True)
    values = merge_df_raw[col].dropna()  
    fig = plt.figure(figsize=(18,9));
    sns.countplot(x=values, color='Sienna', ax = plt.subplot(121));
    sns.boxplot(y=values, color='Sienna', ax = plt.subplot(122));
    plt.suptitle(col, fontsize=16)

In [ ]:
### Categorical variable plots
for col in categories:
    values = merge_df_raw[col].astype('str').value_counts(dropna=False).to_frame().reset_index()
    if len(values) > 30:
        continue
    values.columns = [col, 'counts']
    fig = plt.figure(figsize=(18,9))
    ax = sns.barplot(x=col, y='counts', color='Sienna', data=values, order=values[col]);
    plt.xlabel(col);
    plt.ylabel('Number of occurrences')
    plt.suptitle(col, fontsize=16)

    ### Adding percents over bars
    height = [p.get_height() for p in ax.patches]    
    total = sum(height)
    for i, p in enumerate(ax.patches):    
        ax.text(p.get_x()+p.get_width()/2,
                height[i]+total*0.01,
                '{:1.0%}'.format(height[i]/total),
                ha="center")    

In [ ]:
### Continuous variable vs logerror plots
for col in continuous:     
    fig = plt.figure(figsize=(18,9));
    sns.barplot(x='logerror', y=col, data=merge_df_raw, ax = plt.subplot(121),
                order=['Large Negative Error', 'Medium Negative Error','Small Error',
                       'Medium Positive Error', 'Large Positive Error']);
    plt.xlabel('LogError Bin');
    plt.ylabel('Average {}'.format(col));
    sns.regplot(x='logerror', y=col, data=merge_df_raw, color='Sienna', ax = plt.subplot(122));
    plt.suptitle('LogError vs {}'.format(col), fontsize=16)   

1. Hypothesis : 좋은 전원 주택일수록 데이터가 부족해서 error가 높을 것이다.
> Result : False, 면적이 작을 수록 logerror가 더 컸다.





In [ ]:
df = merge_df_raw.loc[(merge_df_raw['zoning_landuse']!='266.0')]
fig = plt.figure(figsize=(18,9));
sns.barplot(x='logerror', y='area_total_calc', data=df, ax = plt.subplot(121),
            order=['Large Negative Error', 'Medium Negative Error','Small Error',
                   'Medium Positive Error', 'Large Positive Error']);
plt.xlabel('LogError Bin');
plt.ylabel('Average {}'.format(col));
sns.regplot(x='logerror', y='area_total_calc', data=df, color='Sienna', ax = plt.subplot(122));
plt.suptitle('LogError vs {}'.format(col), fontsize=16)   

<a id="11"></a> <br>
# 3.3 Location 데이터

In [ ]:
continuous = ['area_basement','area_firstfloor_finished','area_total_calc','area_live_finished','area_liveperi_finished','area_total_finished','area_unknown','area_base','area_garage','area_lot','area_pool','area_patio','area_shed']
discrete = ['num_bathroom','num_bedroom','num_bathroom_calc','num_bath','num_garage','num_pool','num_room','num_75_bath','num_unit','num_story']
categories = ['aircon','architectural_style','framing','quality','deck','heating','pooltypeid2','pooltypeid7','story','material','build_year']


In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=merge_df_raw.latitude.values, y=merge_df_raw.longitude.values, size=10)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()

In [ ]:
discrete = ['num_fireplace']
categories = ['flag_tub','zoning_landuse_county','zoning_landuse','zoning_property','region_city','region_county','region_neighbor','region_zip','flag_fireplace']

In [ ]:
### Categorical variable plots
for col in categories:
    values = merge_df_raw[col].astype('str').value_counts(dropna=False).to_frame().reset_index()
    if len(values) > 30:
        continue
    values.columns = [col, 'counts']
    fig = plt.figure(figsize=(18,9))
    ax = sns.barplot(x=col, y='counts', color='Sienna', data=values, order=values[col]);
    plt.xlabel(col);
    plt.ylabel('Number of occurrences')
    plt.suptitle(col, fontsize=16)

    ### Adding percents over bars
    height = [p.get_height() for p in ax.patches]    
    total = sum(height)
    for i, p in enumerate(ax.patches):    
        ax.text(p.get_x()+p.get_width()/2,
                height[i]+total*0.01,
                '{:1.0%}'.format(height[i]/total),
                ha="center")    

In [ ]:
### Continuous variable vs logerror plots
for col in continuous:     
    fig = plt.figure(figsize=(18,9));
    sns.barplot(x='logerror', y=col, data=merge_df_raw, ax = plt.subplot(121),
                order=['Large Negative Error', 'Medium Negative Error','Small Error',
                       'Medium Positive Error', 'Large Positive Error']);
    plt.xlabel('LogError Bin');
    plt.ylabel('Average {}'.format(col));
    sns.regplot(x='logerror', y=col, data=merge_df_raw, color='Sienna', ax = plt.subplot(122));
    plt.suptitle('LogError vs {}'.format(col), fontsize=16)   

<a id="13"></a> <br>
# 4. 데이터 변환

In [ ]:
merge_df =  merge_df_raw.copy()

<a id="14"></a> <br>
## 4.1 Null Data 처리

좌표값 없는 데이터 제거

In [ ]:
merge_df = merge_df[np.isfinite(merge_df['latitude'])]
merge_df = merge_df[np.isfinite(merge_df['longitude'])]

In [ ]:
# NULL Check
missing_df = merge_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.ix[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')

ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()

**NULL 다수 값 제거**

'framing', 'area_liveperi_finished', 'area_basement', 'story','area_shed','material','area_base','area_total_finished','area_unknown','num_bathroom_calc','num_bath','area_firstfloor_finished'


**중복 값 제거**

'num_bathroom' : 'num_bathroom_calc','num_bath'


In [ ]:
merge_df = merge_df.drop(['framing', 'area_liveperi_finished', 'area_basement','pooltypeid10','story','area_shed','material','area_base','area_total_finished','area_unknown','area_firstfloor_finished'], axis=1)
merge_df = merge_df.drop(['num_bathroom_calc','num_bath'], axis=1)

**Null 변수 변경**

Null = 0

'flag_tub','pooltypeid2','pooltypeid7','num_pool'

In [ ]:
merge_df = merge_2016_df.append(merge_2017_df, ignore_index=True)
merge_df.shape

In [ ]:
index = merge_df.flag_tub.isnull()
merge_df.loc[index,'flag_tub'] = "None"
# pooltypeid10(does home have a Spa or hot tub) seems to be inconcistent with the 'hashottuborspa' field - these two fields should have the same information I assume?
print(merge_df.flag_tub.value_counts())

#Assume if the pooltype id is null then pool/hottub doesnt exist 
index = merge_df.pooltypeid2.isnull()
merge_df.loc[index,'pooltypeid2'] = 0
print(merge_df.pooltypeid2.value_counts())

index = merge_df.pooltypeid7.isnull()
merge_df.loc[index,'pooltypeid7'] = 0
print(merge_df.pooltypeid7.value_counts())

index = merge_df.num_pool.isnull()
merge_df.loc[index,'num_pool'] = 0
print(merge_df.num_pool.value_counts())

index = merge_df.num_fireplace.isnull()
merge_df.loc[index,'num_fireplace'] = 0
print(merge_df.num_fireplace.value_counts())

#Tax deliquency flag - assume if it is null then doesn't exist
index = merge_df.tax_delinquency.isnull()
merge_df.loc[index,'tax_delinquency'] = "None"
print(merge_df.tax_delinquency.value_counts())


#Assume if Null in garage count it means there are no garages
index = merge_df.num_garage.isnull()
merge_df.loc[index,'num_garage'] = 0
print(merge_df.num_garage.value_counts())

#Likewise no garage means the size is 0 by default
index = merge_df.area_garage.isnull()
merge_df.loc[index,'area_garage'] = 0
print(merge_df.area_garage.value_counts())


다른 변수를 이용해서 Null 값 처리

In [ ]:
#There seems to be inconsistency between the fireplaceflag and fireplace cnt - my guess is that these should be the same
print(merge_df.flag_fireplace.isnull().sum())

#There seems to be 80668 properties without fireplace according to the 'fireplacecnt' but the 'fireplace flag' says they are 90053 missing values
#Lets instead create the fireplaceflag from scratch using 'fireplacecnt' as there are less missing values here
merge_df['flag_fireplace']= "No"
merge_df.loc[merge_df['num_fireplace']>0,'flag_fireplace']= "Yes"
print(merge_df.flag_fireplace.isnull().sum())

빈도수에 따른 Null 값 처리

In [ ]:
#Let's fill in some missing values using the most common value for those variables where this might be a sensible approach
#AC Type - Mostly 1's, which corresponds to central AC. Reasonable to assume most other properties are similar.
merge_df['aircon'].value_counts()
index = merge_df.aircon.isnull()
merge_df.loc[index,'aircon'] = 1
print(merge_df.aircon.value_counts())

#heating or system - Mostly 2, which corresponds to central heating so seems reasonable to assume most other properties have central heating  
print(merge_df['heating'].value_counts())
index = merge_df.heating.isnull()
merge_df.loc[index,'heating'] = 2

# 'threequarterbathnbr' - not an important variable according to https://www.kaggle.com/nikunjm88/creating-additional-features, so fill with most common value
print(merge_df['num_75_bath'].value_counts())
index = merge_df.num_75_bath.isnull()
merge_df.loc[index,'num_75_bath'] = 1

In [ ]:
merge_df.shape

In [ ]:
missingvalues_prop = (merge_df.isnull().sum()/len(merge_df)).reset_index()
missingvalues_prop.columns = ['field','proportion']
missingvalues_prop = missingvalues_prop.sort_values(by = 'proportion', ascending = False)
print(missingvalues_prop)
missingvaluescols = missingvalues_prop[missingvalues_prop['proportion'] > 0.97].field.tolist()
merge_df = merge_df.drop(missingvaluescols, axis=1)

위치 기반 Null 처리

- Null 값을 가진 부동산의 경우 주변 부동산과 비슷한 특성을 갖고 있을 것이다. 
>Target 부동산의 주변 부동산을 KNN으로 분석해서 null 처리 한다.

In [ ]:
def fillna_knn2( df, base, target, fraction = 1, threshold = 10, n_neighbors = 5 ):
    assert isinstance( base , list ) or isinstance( base , np.ndarray ) and isinstance( target, str ) 
    whole = [ target ] + base
    
    miss = df[target].isnull()
    notmiss = ~miss 
    nummiss = miss.sum()
    
    enc = OneHotEncoder()
    X_target = df.loc[ notmiss, whole ].sample( frac = fraction )
    
    enc.fit( X_target[ target ].unique().reshape( (-1,1) ) )
    
    Y = enc.transform( X_target[ target ].values.reshape((-1,1)) ).toarray()
    X = X_target[ base  ]
    
    print( 'fitting' )
    n_neighbors = n_neighbors
    clf = neighbors.KNeighborsClassifier( n_neighbors, weights = 'uniform' )
    clf.fit( X, Y )
    
    print( 'the shape of active features: ' ,enc.active_features_.shape )
    
    print( 'predicting' )
    Z = clf.predict(df.loc[miss, base])
    
    numunperdicted = Z[:,0].sum()
    if numunperdicted / nummiss *100 < threshold :
        print( 'writing result to df' )    
        df.loc[ miss, target ]  = np.dot( Z , enc.active_features_ )
        print( 'num of unperdictable data: ', numunperdicted )
        return enc
    else:
        print( 'out of threshold: {}% > {}%'.format( numunperdicted / nummiss *100 , threshold ) )

#function to deal with variables that are actually string/categories
def zoningcode2int( df, target ):
    storenull = df[ target ].isnull()
    enc = LabelEncoder( )
    df[ target ] = df[ target ].astype( str )

    print('fit and transform')
    df[ target ]= enc.fit_transform( df[ target ].values )
    print( 'num of categories: ', enc.classes_.shape  )
    df.loc[ storenull, target ] = np.nan
    print('recover the nan value')
    return enc

In [ ]:
#buildingqualitytypeid - assume it is the similar to the nearest property. Probably makes senses if its a property in a block of flats, i.e if block was built all at the same time and therefore all flats will have similar quality 
#Use the same logic for propertycountylandusecode (assume it is same as nearest property i.e two properties right next to each other are likely to have the same code) & propertyzoningdesc. 
#These assumptions are only reasonable if you actually have nearby properties to the one with the missing value
'''
fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'quality', fraction = 0.15, n_neighbors = 1 )


zoningcode2int( df = merge_df,
                            target = 'zoning_landuse_county' )
fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'zoning_landuse_county', fraction = 0.15, n_neighbors = 1 )

zoningcode2int( df = merge_df,
                            target = 'zoning_property' )

fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'zoning_property', fraction = 0.15, n_neighbors = 1 )

#regionidcity, regionidneighborhood & regionidzip - assume it is the same as the nereast property. 
#As mentioned above, this is ok if there's a property very nearby to the one with missing values (I leave it up to the reader to check if this is the case!)
fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'region_city', fraction = 0.15, n_neighbors = 1 )

fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'region_neighbor', fraction = 0.15, n_neighbors = 1 )

fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'region_zip', fraction = 0.15, n_neighbors = 1 )

#unitcnt - the number of structures the unit is built into. Assume it is the same as the nearest properties. If the property with missing values is in a block of flats or in a terrace street then this is probably ok - but again I leave it up to the reader to check if this is the case!
fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'num_unit', fraction = 0.15, n_neighbors = 1 )

#yearbuilt - assume it is the same as the nearest property. This assumes properties all near to each other were built around the same time
fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'build_year', fraction = 0.15, n_neighbors = 1 )

#lot size square feet - not sure what to do about this one. Lets use nearest neighbours. Assume it has same lot size as property closest to it
fillna_knn( df = merge_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'area_lot', fraction = 0.15, n_neighbors = 1 )
'''

In [ ]:
merge_df.to_csv('merge_df.csv',index=False)

References

https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-zillow-prize
https://www.kaggle.com/neviadomski/zillow-s-home-value-exploratory-data-analysis
https://www.kaggle.com/kueipo/simple-eda-geo-data-time-series
